# Import libraries and define a few variables

In [1]:
import matplotlib.pyplot as plt
import ecg_plot
import numpy as np
import os 
import neurokit2 as nk
import random
import warnings

from scipy.io import loadmat
from glob import glob

warnings.filterwarnings('ignore')

In [2]:
path_mat = glob('Training_2/*.mat')

In [3]:
len(path_mat)

3453

# Helper Functions

In [4]:
def load_data(filename,calib):
    
    x = loadmat(filename)
    data = np.asarray(x['val'], dtype=np.float64)
        
    lead_2 = data[1]/calib
    lead_3 = data[2]/calib

    avf = data[5]/calib

    v2 = data[7]/calib
    v3 = data[8]/calib
    v4 = data[9]/calib
    v5 = data[10]/calib
    v6 = data[11]/calib
    
    new_file = filename.replace('.mat','.hea')
    input_header_file = os.path.join(new_file)
    with open(input_header_file,'r') as f:
        header_data=f.readlines()
        
    gender = header_data[-5][6:].split('\n')[0]
    
    dict_1 = {'lead_2': lead_2,
              'lead_3': lead_3,
               'avf': avf,
                'v2': v2,
            'v3': v3,
             'v4': v4,
             'v5': v5,
             'v6': v6} 
    
    return dict(dict_1) ,gender,header_data

In [5]:
def calculate_ST_elevation(st_segment,baseline_val):
    
    # Calculate the maximum value in the ST segment
    st_max = np.max(st_segment)
    
    # Calculate the ST segment elevation as the difference between the baseline and maximum values
    st_elevation = st_max - baseline_val
    
    return st_elevation

# Contents of the any one of the files

In [18]:
n = random.randint(0,len(path_mat))

In [19]:
x = loadmat(path_mat[n])
data = np.asarray(x['val'], dtype=np.float64)
    
new_file = path_mat[n].replace('.mat','.hea')
input_header_file = os.path.join(new_file)
with open(input_header_file,'r') as f:
    header_data=f.readlines()

In [20]:
data.shape #consists voltage vals from all the leads

(12, 5000)

In [21]:
data 

array([[ 66.,  97.,  96., ..., 263., 248., 274.],
       [117., 171., 167., ..., 437., 415., 457.],
       [ 50.,  74.,  71., ..., 174., 167., 183.],
       ...,
       [247., 362., 347., ..., 805., 754., 817.],
       [209., 311., 301., ..., 722., 675., 733.],
       [186., 274., 261., ..., 480., 450., 494.]])

In [22]:
header_data 

['Q2503 12 500 5000 15-May-2020 15:35:30\n',
 'Q2503.mat 16+24 1000/mV 16 0 66 -68 0 I\n',
 'Q2503.mat 16+24 1000/mV 16 0 117 -94 0 II\n',
 'Q2503.mat 16+24 1000/mV 16 0 50 -40 0 III\n',
 'Q2503.mat 16+24 1000/mV 16 0 -92 66 0 aVR\n',
 'Q2503.mat 16+24 1000/mV 16 0 8 7 0 aVL\n',
 'Q2503.mat 16+24 1000/mV 16 0 84 -36 0 aVF\n',
 'Q2503.mat 16+24 1000/mV 16 0 67 6 0 V1\n',
 'Q2503.mat 16+24 1000/mV 16 0 328 -98 0 V2\n',
 'Q2503.mat 16+24 1000/mV 16 0 279 -155 0 V3\n',
 'Q2503.mat 16+24 1000/mV 16 0 247 -117 0 V4\n',
 'Q2503.mat 16+24 1000/mV 16 0 209 -109 0 V5\n',
 'Q2503.mat 16+24 1000/mV 16 0 186 -101 0 V6\n',
 '#Age: 23\n',
 '#Sex: Male\n',
 '#Dx: 164865005,164931005\n',
 '#Rx: Unknown\n',
 '#Hx: Unknown\n',
 '#Sx: Unknown\n']

## Explanation of header data:



### Q2503 12 500 5000 15-May-2020 15:35:30:

This line indicates general information about the ECG recording, including:
- the patient identifier (Q2503), 
- the number of leads (12), 
- the sampling rate (500 samples per second), 
- the number of samples (5000), and 
- the date and time of the recording.



### Q2503.mat 16+24 1000/mV 16 0 66 -68 0 I:

This line represents lead I of the ECG recording. The values indicate :
- the file name ('Q2503.mat'), 
- the gain (16+24), 
- the calibration (1000/mV), 
- the baseline (16), and 
- the voltage values for each sample point.

<b>Note:</b>: Its the same for other 11 leads too

### #Age: 23': 
This line indicates the age of the patient (23 years old).

### #Sex: Male: 
This line indicates the gender of the patient (Male).

### #Dx: 164865005,164931005: 
This line indicates the diagnosis codes associated with the patient's condition. The specific codes provided here are not explained in the given data.

### #Rx: Unknown: 
This line indicates that the information about the patient's prescribed medications is unknown.

### #Hx: Unknown: 
This line indicates that the information about the patient's medical history is unknown.

### #Sx: Unknown: 
This line indicates that the information about the patient's symptoms is unknown.

# Steps followed:

## Step 1:

Extracted only the relevant lead information from the raw ecg

## Step 2:

Extracted R peaks for further extraction of S peaks and T onsets 

## Step 3:

Calculated the ST segment using the S peaks and T onset

## Step 4:

Derived the elevation of the St segment by comparing the max value in ST segment with the baseline value

## Step 5:

If the ST elevation >= 1 mm for male and 0.5 mm for female, check if 2 or more contiguous leads portray such behaviour. If yes, consider that as a possibility for MI

# Implementation of the above steps

## Defining a few fixed variables

In [6]:
#these vals are the same for this dataset, hence defining it already, if not read it from header_data

baseline = 16
sr = 500
calib = 1000
cont_1 = set(['v1','v2','v3','v4','v5','v6']) #contiguous leads (limb)
cont_2 = set(['lead_2','lead_3','avf']) #contiguous leads (chest)

## Actual Calculation

In [7]:
m = random.randint(0,len(path_mat)-11)

In [8]:
for i in range(m,m+10):
    count = 0
    leads = []
    ecg = path_mat[i]
    print(f'Calculating ST elevation for {ecg}')
    extracted_data = load_data(ecg,calib)
    gender = extracted_data[-2]

    for lead in extracted_data[0].keys():

        _, rpeaks = nk.ecg_peaks(extracted_data[0][lead], sampling_rate=sr) #calculate R-peaks
        _, waves_peak = nk.ecg_delineate(extracted_data[0][lead], 
                                 rpeaks, 
                                 sampling_rate=sr, 
                                 method="dwt", 
                                 show=False, 
                                 show_type='all')
        
        for s,t in zip(waves_peak['ECG_S_Peaks'],waves_peak['ECG_T_Onsets']):
            if str(s) != 'nan' and str(t) != 'nan':
                st_segment_values = extracted_data[0][lead][baseline: t]
                
                if st_segment_values.shape[0] > 0:
                    st_elevation = calculate_ST_elevation(st_segment_values,extracted_data[0][lead][baseline])
                    break
                else:
                    pass
            else:
                pass
  
        if st_elevation >= 0.1 and gender == 'Male':
            count+=1
            leads.append(lead)
        elif st_elevation >= 0.05 and gender == 'Female':
            count+=1
            leads.append(lead)
        else:
            pass
        
    leads = set(leads)    
    if len(cont_1.intersection(leads)) >= 2:
        print(f'ST for {cont_1.intersection(leads)} leads greater than the threshold for {gender} ! Might be MI.')
        print()
    elif len(cont_2.intersection(leads)) >=2:
        print(f'ST for {cont_2.intersection(leads)} leads greater than the threshold for {gender} ! Might be MI.')
        print()
    else:
        print(f'insignificant ST ({cont_1.intersection(leads)}) for {gender}. No possibilities of MI.')
        print()

Calculating ST elevation for Training_2\Q0325.mat
ST for {'v5', 'v3', 'v2', 'v6', 'v4'} leads greater than the threshold for Male ! Might be MI.

Calculating ST elevation for Training_2\Q0326.mat
ST for {'v5', 'v3', 'v2', 'v6', 'v4'} leads greater than the threshold for Female ! Might be MI.

Calculating ST elevation for Training_2\Q0327.mat
ST for {'v5', 'v3', 'v2', 'v6', 'v4'} leads greater than the threshold for Female ! Might be MI.

Calculating ST elevation for Training_2\Q0328.mat
ST for {'v5', 'v3', 'v2', 'v6', 'v4'} leads greater than the threshold for Female ! Might be MI.

Calculating ST elevation for Training_2\Q0329.mat
ST for {'v5', 'v6', 'v3', 'v4'} leads greater than the threshold for Male ! Might be MI.

Calculating ST elevation for Training_2\Q0330.mat
ST for {'v5', 'v3', 'v2', 'v6', 'v4'} leads greater than the threshold for Male ! Might be MI.

Calculating ST elevation for Training_2\Q0331.mat
ST for {'v5', 'v6', 'v3', 'v4'} leads greater than the threshold for Male 